# **Práctica 2: Clasificación Multi-Instancia**

## Opción 2: Comparación de métodos

**Objetivo**: Comparar al menos dos algoritmos disponibles en bibliotecas de aprendizaje multi-instancia. Se seleccionan al menos tres conjuntos de datos de tipo multi-instancia para realizar la evaluación y comparación.

En este notebook:
- Se entrenan dos modelos de clasificación multi-instancia: **MISVM** y **MILES**.
- Se usan tres datasets: **Musk1**, **Elephant** y **Corel Dogs**.
- Se comparan sus resultados (precisión, F1, AUC y matrices de confusión).
- Se presentan conclusiones sobre los hallazgos.

## 1. Instalación de librerías
Si usas Google Colab, puedes necesitar instalar las librerías correspondientes. Si ya tienes un entorno con `misvm` y `mil` instalados, puedes omitirlo.

In [ ]:
!pip install git+https://github.com/garydoranjr/misvm.git
!pip install numpy scikit-learn scipy tensorflow==2.12.0 mil

## 2. Importaciones y carga de datos
Se importan las librerías necesarias y se cargan los datasets **Musk1**, **Elephant** y **Corel Dogs** a partir de la biblioteca `mil`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

import misvm
from mil.data.datasets import loader
from mil.bag_representation import MILESMapping
from mil.validators import LeaveOneOut
from mil.models import SVC
from mil.trainer import Trainer
from mil.preprocessing import StandarizerBagsList
from mil.metrics import AUC

print("Cargando datasets Musk1, Elephant y Corel Dogs...")
musk1_train, musk1_test = loader.load_data('/usr/local/lib/python3.11/dist-packages/mil/data/datasets/csv/musk1.csv')
elephant_train, elephant_test = loader.load_data('/usr/local/lib/python3.11/dist-packages/mil/data/datasets/csv/elephant.csv')
corel_train, corel_test = loader.load_data('/usr/local/lib/python3.11/dist-packages/mil/data/datasets/csv/corel_dogs.csv')

musk1_bags_train, musk1_y_train = musk1_train
musk1_bags_test, musk1_y_test = musk1_test

elephant_bags_train, elephant_y_train = elephant_train
elephant_bags_test, elephant_y_test = elephant_test

corel_bags_train, corel_y_train = corel_train
corel_bags_test, corel_y_test = corel_test

print("Datos cargados correctamente.")

## 3. Definición de funciones de evaluación
En esta sección, definimos:

- `evaluate_misvm(...)`: para entrenar y evaluar un clasificador **MISVM** (de la librería [garydoranjr/misvm](https://github.com/garydoranjr/misvm)).
- `evaluate_miles(...)`: para entrenar y evaluar un clasificador **MILES** usando la biblioteca [`mil`](https://pypi.org/project/mil/).

In [ ]:
def evaluate_misvm(bags_train, y_train, bags_test, y_test, kernel='linear', C=1.0, max_iters=50):
    """
    Entrena y evalúa un clasificador MISVM.
    Retorna (accuracy, f1_score, matriz_confusion).
    """
    print(f"Entrenando MISVM (kernel={kernel}, C={C}, max_iters={max_iters})...")
    classifier = misvm.MISVM(kernel=kernel, C=C, max_iters=max_iters)
    y_train_binary = np.array([1 if y > 0 else 0 for y in y_train])
    y_test_binary = np.array([1 if y > 0 else 0 for y in y_test])

    classifier.fit(bags_train, y_train_binary)
    y_pred = classifier.predict(bags_test)

    y_pred_binary = np.array([1 if p > 0 else 0 for p in y_pred])

    acc = accuracy_score(y_test_binary, y_pred_binary)
    f1 = f1_score(y_test_binary, y_pred_binary, zero_division=0)
    cm = confusion_matrix(y_test_binary, y_pred_binary)

    return acc, f1, cm

def evaluate_miles(bags_train, y_train, bags_test, y_test, kernel='linear', C=1.0):
    """
    Entrena y evalúa MILES utilizando el pipeline de la librería mil.
    Retorna (accuracy, auc).
    """
    print(f"Entrenando MILES (kernel={kernel}, C={C})...")
    trainer = Trainer()
    metrics = ['acc', AUC]
    model = SVC(kernel=kernel, C=C, class_weight='balanced')
    pipeline = [
        ('scale', StandarizerBagsList()),
        ('disc_mapping', MILESMapping())
    ]
    trainer.prepare(model, preprocess_pipeline=pipeline, metrics=metrics)

    y_train_binary = np.array([1 if y > 0 else 0 for y in y_train])
    y_test_binary = np.array([1 if y > 0 else 0 for y in y_test])

    valid = LeaveOneOut()

    trainer.fit(bags_train, y_train_binary, sample_weights='balanced', validation_strategy=valid, verbose=0)

    result = trainer.predict_metrics(bags_test, y_test_binary)
    acc = result.get('acc', 0.0)
    auc_ = result.get('auc', 0.5)

    return acc, auc_


## 4. Ejecución de experimentos
Se evalúan **MISVM** y **MILES** en cada uno de los tres datasets. Al final se grafican las matrices de confusión y se compara el rendimiento de ambos métodos en cada problema.

In [ ]:
print("\n=== Evaluando algoritmos en datasets: Musk1, Elephant, Corel Dogs ===")

misvm_params = {
    'kernel': 'rbf',
    'C': 10.0,
    'max_iters': 50
}
miles_params = {
    'kernel': 'linear',
    'C': 1.0
}

## MISVM ##
misvm_musk1 = evaluate_misvm(musk1_bags_train, musk1_y_train, musk1_bags_test, musk1_y_test,
                             kernel=misvm_params['kernel'], C=misvm_params['C'], max_iters=misvm_params['max_iters'])
misvm_elephant = evaluate_misvm(elephant_bags_train, elephant_y_train, elephant_bags_test, elephant_y_test,
                                kernel=misvm_params['kernel'], C=misvm_params['C'], max_iters=misvm_params['max_iters'])
misvm_corel = evaluate_misvm(corel_bags_train, corel_y_train, corel_bags_test, corel_y_test,
                             kernel=misvm_params['kernel'], C=misvm_params['C'], max_iters=misvm_params['max_iters'])

## MILES ##
miles_musk1 = evaluate_miles(musk1_bags_train, musk1_y_train, musk1_bags_test, musk1_y_test,
                             kernel=miles_params['kernel'], C=miles_params['C'])
miles_elephant = evaluate_miles(elephant_bags_train, elephant_y_train, elephant_bags_test, elephant_y_test,
                                kernel=miles_params['kernel'], C=miles_params['C'])
miles_corel = evaluate_miles(corel_bags_train, corel_y_train, corel_bags_test, corel_y_test,
                             kernel=miles_params['kernel'], C=miles_params['C'])

print("\n=== Resultados MISVM ===")
print(f"Musk1 -> Accuracy: {misvm_musk1[0]:.4f}, F1: {misvm_musk1[1]:.4f}")
print(f"Elephant -> Accuracy: {misvm_elephant[0]:.4f}, F1: {misvm_elephant[1]:.4f}")
print(f"Corel Dogs -> Accuracy: {misvm_corel[0]:.4f}, F1: {misvm_corel[1]:.4f}")

print("\n=== Resultados MILES ===")
print(f"Musk1 -> Accuracy: {miles_musk1[0]:.4f}, AUC: {miles_musk1[1]:.4f}")
print(f"Elephant -> Accuracy: {miles_elephant[0]:.4f}, AUC: {miles_elephant[1]:.4f}")
print(f"Corel Dogs -> Accuracy: {miles_corel[0]:.4f}, AUC: {miles_corel[1]:.4f}")

## 5. Visualización de matrices de confusión (MISVM)
Para MILES, calculamos la métrica AUC, pero no obtenemos la matriz de confusión directa en la misma función. Puedes modificar la evaluación de MILES para obtener y mostrar la matriz de confusión si se desea.

In [ ]:
def plot_confusion_matrix(cm, title):
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

print("\n=== Matrices de confusión para MISVM ===")
plot_confusion_matrix(misvm_musk1[2], 'Musk1 - MISVM')
plot_confusion_matrix(misvm_elephant[2], 'Elephant - MISVM')
plot_confusion_matrix(misvm_corel[2], 'Corel Dogs - MISVM')

## 6. Comparación gráfica de la **Accuracy**
Se crea una gráfica de barras para comparar la precisión de **MISVM** y **MILES** en cada dataset.

In [ ]:
def plot_comparison(datasets, misvm_metrics, miles_metrics, metric_name='Accuracy'):
    """Crea un diagrama de barras comparando la métrica dada para MISVM y MILES."""
    misvm_values = [m[0] for m in misvm_metrics]
    miles_values = [m[0] for m in miles_metrics]

    x = np.arange(len(datasets))
    width = 0.35

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.bar(x - width/2, misvm_values, width, label='MISVM')
    ax.bar(x + width/2, miles_values, width, label='MILES')

    ax.set_ylabel(metric_name)
    ax.set_title(f'{metric_name} por algoritmo y dataset')
    ax.set_xticks(x)
    ax.set_xticklabels(datasets)
    ax.set_ylim([0, 1])
    ax.legend()
    plt.show()

datasets = ['Musk1', 'Elephant', 'Corel']
misvm_results = [misvm_musk1, misvm_elephant, misvm_corel]
miles_results = [miles_musk1, miles_elephant, miles_corel]

plot_comparison(datasets, misvm_results, miles_results, 'Accuracy')

## 7. Conclusiones
- Hemos comparado dos métodos de clasificación multi-instancia:
  - **MISVM** (paradigma basado en SVM a nivel de bolsas)
  - **MILES** (basado en mapeo de instancias y SVM).
- Se han utilizado tres conjuntos de datos clásicos en MIL: **Musk1**, **Elephant** y **CorelDogs**.
- Los resultados muestran diferentes rendimientos en cada dataset. Dependiendo de la complejidad de las bolsas y de la representación, un método puede superar al otro.
- **MILES** aporta la ventaja de un mapeo de instancias a un espacio de instancia prototipo, mientras que **MISVM** aprende directamente con las bolsas.
- En general, se observa que el comportamiento varía por la naturaleza de los datos. Algunos datasets (p.ej., Musk1) parecen más sencillos para ambos métodos y muestran una mayor **Accuracy**.
- Se podrían ajustar más los hiperparámetros (p.ej., tipo de kernel, regularización **C**, etc.) para mejorar los resultados y afinar la comparación.

En conclusión, ambos métodos son válidos para problemas de clasificación multi-instancia, pero su rendimiento depende en buena medida de:
1. El **tipo de dataset** (características, número de instancias por bolsa, complejidad de la distribución).
2. Los **hiperparámetros** escogidos (kernel, C, etc.).
3. Técnicas de **preprocesamiento** o **selección de características**.